In [ ]:
import sys
sys.path.append('../app/')
from psa import Strategy, Disease, Psa, Icer
from load_data import data_loader

In [ ]:
import os

#file_disease = '../data/Dati Input Disease.xlsx'
#file_strategies = '../data/Dati Input Strategies.xlsx'
#file_costs = '../data/Dati Input Costs.xlsx'
#out_folder = '../output/igenomix_af'

file_disease = '../data/Dati Input Disease_betathalassemia.xlsx'
file_strategies = '../data/Dati Input Strategies.xlsx'
file_costs = '../data/Dati Input Strategy Costs.xlsx'
out_folder = '../output/betathalassemia'


os.makedirs('../output_v2',exist_ok=True)
os.makedirs(out_folder,exist_ok=True)

# Toy data

In [ ]:
# Disease Definitions
d1 = Disease(name='disease1',carrier_rate=0.01,life_expectancy=80,cost=10000)
d2 = Disease(name='disease2',carrier_rate=0.02,life_expectancy=60,cost=20000)
d3 = Disease(name='disease3',carrier_rate=0.005,life_expectancy=40,cost=100000)
disease_dict_fake = {'disease1':d1,'disease2':d2,'disease3':d3}


# Strategies Definition
s0 = Strategy('No testing',disease_list=[],testing_cost=0,disease_dict=disease_dict_fake)
s1 = Strategy('Limited screening',disease_list=['disease2'],testing_cost=1000,disease_dict=disease_dict_fake)
s2 = Strategy('Full screening',disease_list=['disease1','disease2','disease3'],testing_cost=1200,disease_dict=disease_dict_fake)

strategy_dict_fake = {'s0':s0,'s1':s1,'s2':s2}




In [ ]:
s0.assess_strategy(disease_space=['disease1','disease2','disease3'])

In [ ]:
icer1 = Icer(s2,s1)
icer1.compute_icer()
display(icer1.icer)
display(icer1.delta_years)
display(icer1.delta_costs)

In [ ]:
icer1.univariate_sensitivity('testing_cost1',0,12000,50)
icer1.plot_sensitivity()

In [ ]:
psa = Psa(s2,s0)
psa.run_mc(disease_space=['disease1','disease2','disease3'],
    intervention_cost_lb=2000, intervention_cost_ub=3000,
       testing_cost1_lb=1000, testing_cost1_ub=1300,
        testing_cost2_lb=0, testing_cost2_ub=0,
        eps_cs_lb=0,eps_cs_ub=0.02,
        eps_pgt_lb=0,eps_pgt_ub=0.02,
        rho_notint_lb=0.1,rho_notint_ub=0.4,nsim=10000)

psa.plot_ceac_hist()

# Load real data

In [ ]:


dl = data_loader(file_disease,file_strategies,file_costs=file_costs)



In [ ]:
dl.disease_dict['Beta Thalassemia'].__str__()

In [ ]:
dl.disease_dict['Beta Thalassemia']

In [ ]:
for strategy in dl.strategy_dict.keys():

    display(strategy,
            dl.strategy_dict[strategy].assess_strategy(intervention_cost=19714,eps_cs=0,eps_pgt=0))

# cost effectivness plane

In [ ]:
dl.strategy_dict.keys()

In [ ]:
import pandas as pd
df_icer = pd.DataFrame()

for comparison in [
                  #('Strategy A1 (MS)','No testing'),
                  ('Strategy A (FS)','No testing'),
                  #('Strategy A1 (MS)','Strategy C (ACOG)'),
                  #('Strategy A1 (MS)','Strategy D (ACMG)'),
                  #('Strategy A (FS)','Strategy C (ACOG)'),
                  #('Strategy A (FS)','Strategy D (ACMG)'),
                  #('Strategy A (FS)','Strategy A1 (MS)'),
                   ('Strategy A (FS)','Strategy B (ACOG+ACMG)'),
                    ('Strategy B (ACOG+ACMG)','No testing'),
                  ]:
    strategy1 = comparison[0]
    strategy2 = comparison[1]

    icer1 = Icer(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2],
                 eps_cs=0,eps_pgt=0,intervention_cost=19714)
    icer1.compute_icer()
    #display(icer1.disease_space)
    df_icer = df_icer.append(
        {'strategy1':strategy1,'strategy2':strategy2,
           'icer':icer1.icer,
           'total cost1':icer1.total_cost1,
           'total cost2':icer1.total_cost2,
           'life exp1':icer1.life_exp1,
           'life exp2':icer1.life_exp2,
          'benefit difference (years)':icer1.life_exp1-icer1.life_exp2,
        'cost difference (Euros)':icer1.total_cost1-icer1.total_cost2},ignore_index=True)

df_icer['cfr'] = df_icer['strategy1']+' vs '+df_icer['strategy2']

In [ ]:
df_icer.columns

In [ ]:


df_icer

In [ ]:
df_icer[['strategy1','strategy2','icer','benefit difference (years)', 'cost difference (Euros)','life exp1', 'life exp2','total cost1',
       'total cost2']].to_excel(os.path.join(out_folder,'icer.xlsx'))

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

fig,ax = plt.subplots(figsize=(11,7))

sns.scatterplot(ax=ax,data = df_icer,x='benefit difference (years)',y='cost difference (Euros)',color='r',s=50)

plt.axhline(0,c='k')
plt.axvline(0,c='k')
plt.xlim([-0.03,0.03])
plt.ylim([-2000,2000])

icer_th = 50000
x1 = -0.03
x2 = 0.03
plt.plot([x1,x2],[x1*icer_th,x2*icer_th],ls='--',c='r',label='50.000 Euros/LYG')

for line in range(0,df_icer.shape[0]):
     plt.text(df_icer['benefit difference (years)'][line],
              df_icer['cost difference (Euros)'][line], 
              df_icer['cfr'][line], horizontalalignment='right', verticalalignment='bottom', size='medium', color='blue')#, weight='semibold')
plt.legend()
plt.grid()
plt.savefig(os.path.join(out_folder,'cost_effectivness_plane.jpeg'))

# Sensitivity analysis

In [ ]:
import pandas as pd

intervention_cost = 19714
rho_notint = 0.23
MS_cost = dl.strategy_dict['Strategy A1 (MS)'].testing_cost
FS_cost = dl.strategy_dict['Strategy A (FS)'].testing_cost
ACMG_cost = dl.strategy_dict['Strategy D (ACMG)'].testing_cost
ACOG_cost = dl.strategy_dict['Strategy C (ACOG)'].testing_cost
ACOGACMG_cost = dl.strategy_dict['Strategy B (ACOG+ACMG)'].testing_cost

for strategy in dl.strategy_dict.keys():
    print(strategy,dl.strategy_dict[strategy].testing_cost)

In [ ]:
# FS vs MS 


strategy1 = 'Strategy A (FS)'
strategy2 = 'Strategy A1 (MS)'


param_list = [
    ('testing_cost2',0.75*MS_cost,'MS cost -25%'),
              ('testing_cost2',1.25*MS_cost,'MS cost +25%'),
              ('testing_cost1',0.75*FS_cost,'FS cost -25%'),
              ('testing_cost1',1.25*FS_cost,'FS cost +25%'),
              ('intervention_cost',1.25*intervention_cost,'Intervention cost +25%'),
              ('intervention_cost',0.75*intervention_cost,'Intervention cost -25%'),
                ('rho_notint',0.4,'Intervention rate 60%'),
              ('rho_notint',0.1,'Intervention rate 90%'),
             ('eps_cs',0.02,'ECS detection rate 98%'),
             ('eps_pgt',0.02,'Intervention success rate 98%'),
    ('original_model_fake_parameter',0,'Original Model')]

df_icer_sensitivity = pd.DataFrame()

for param1 in param_list:
    for param2 in param_list:
        
        if (param1[1]!=param2[1])&(param1[0]==param2[0]):
            pass
        
        else:

            #print(param1,param2)

            icer = Icer(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2],
                     eps_cs=0,eps_pgt=0,intervention_cost=intervention_cost,rho_notint=rho_notint)
            setattr(icer,param1[0],param1[1])
            setattr(icer,param2[0],param2[1])


            icer.compute_icer()
            df_icer_sensitivity = df_icer_sensitivity.append(
            {'param1':param1[2],'param1_value':param1[1],
             'param2':param2[2],'param2_value':param2[1],
               'icer':icer.icer,
               'total cost1':icer.total_cost1,
               'total cost2':icer.total_cost2,
               'life exp1':icer.life_exp1,
               'life exp2':icer.life_exp2,
              'benefit difference (years)':icer.life_exp1-icer.life_exp2,
            'cost difference (Euros)':icer.total_cost1-icer.total_cost2},ignore_index=True)

df_pivot = df_icer_sensitivity.pivot_table(index='param1',columns='param2',values='icer')
params = [x[2] for x in param_list]
df_pivot = df_pivot.loc[params,params]

fig,ax = plt.subplots(figsize=(12,8))
sns.heatmap(ax=ax,data = df_pivot, annot=True, fmt=".0f")
plt.savefig(os.path.join(out_folder,'bivariate_'+strategy1+'vs'+strategy2+'.jpeg'))

In [ ]:
# MS vs no testing


strategy1 = 'Strategy A1 (MS)'
strategy2 = 'No testing'

param_list = [
    ('testing_cost1',0.75*MS_cost,'MS cost -25%'),
              ('testing_cost1',1.25*MS_cost,'MS cost +25%'),
              ('intervention_cost',1.25*intervention_cost,'Intervention cost +25%'),
              ('intervention_cost',0.75*intervention_cost,'Intervention cost -25%'),
                ('rho_notint',0.4,'Intervention rate 60%'),
              ('rho_notint',0.1,'Intervention rate 90%'),
             ('eps_cs',0.02,'ECS detection rate 98%'),
             ('eps_pgt',0.02,'Intervention success rate 98%'),
    ('original_model_fake_parameter',0,'Original Model')]


df_icer_sensitivity = pd.DataFrame()

for idx1 in range(len(param_list)):
    for idx2 in range(len(param_list)):
        
        param1 = param_list[idx1]
        param2 = param_list[idx2]
        #print(param1,param2)

        
        if (param1[1]!=param2[1])&(param1[0]==param2[0]):
            pass
        
        else:
        
        
            icer = Icer(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2],
                     eps_cs=0,eps_pgt=0,intervention_cost=intervention_cost,rho_notint=rho_notint)
            setattr(icer,param1[0],param1[1])
            setattr(icer,param2[0],param2[1])


            icer.compute_icer()
            df_icer_sensitivity = df_icer_sensitivity.append(
            {'param1':param1[2],'param1_value':param1[1],
             'param2':param2[2],'param2_value':param2[1],
               'icer':icer.icer,
               'total cost1':icer.total_cost1,
               'total cost2':icer.total_cost2,
               'life exp1':icer.life_exp1,
               'life exp2':icer.life_exp2,
              'benefit difference (years)':icer.life_exp1-icer.life_exp2,
            'cost difference (Euros)':icer.total_cost1-icer.total_cost2},ignore_index=True)

df_pivot = df_icer_sensitivity.pivot_table(index='param1',columns='param2',values='icer')
params = [x[2] for x in param_list]
#df_pivot = df_pivot.loc[params,params]

fig,ax = plt.subplots(figsize=(12,8))
sns.heatmap(ax=ax,data = df_pivot, annot=True, fmt=".0f")
plt.savefig(os.path.join(out_folder,'bivariate_'+strategy1+'vs'+strategy2+'.jpeg'))

In [ ]:
# FS vs no testing


strategy1 = 'Strategy A (FS)'
strategy2 = 'No testing'

param_list = [
             ('testing_cost1',0.75*FS_cost,'FS cost -25%'),
              ('testing_cost1',1.25*FS_cost,'FS cost +25%'),
              ('intervention_cost',1.25*intervention_cost,'Intervention cost +25%'),
              ('intervention_cost',0.75*intervention_cost,'Intervention cost -25%'),
                ('rho_notint',0.4,'Intervention rate 60%'),
              ('rho_notint',0.1,'Intervention rate 90%'),
             ('eps_cs',0.02,'ECS detection rate 98%'),
             ('eps_pgt',0.02,'Intervention success rate 98%'),
    ('original_model_fake_parameter',0,'Original Model')]


df_icer_sensitivity = pd.DataFrame()

for idx1 in range(len(param_list)):
    for idx2 in range(len(param_list)):
        
        param1 = param_list[idx1]
        param2 = param_list[idx2]
        
        if (param1[1]!=param2[1])&(param1[0]==param2[0]):
            pass
        
        else:
        
            #print(param1,param2)

            icer = Icer(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2],
                     eps_cs=0,eps_pgt=0,intervention_cost=intervention_cost,rho_notint=rho_notint)
            setattr(icer,param1[0],param1[1])
            setattr(icer,param2[0],param2[1])


            icer.compute_icer()
            df_icer_sensitivity = df_icer_sensitivity.append(
            {'param1':param1[2],'param1_value':param1[1],
             'param2':param2[2],'param2_value':param2[1],
               'icer':icer.icer,
               'total cost1':icer.total_cost1,
               'total cost2':icer.total_cost2,
               'life exp1':icer.life_exp1,
               'life exp2':icer.life_exp2,
              'benefit difference (years)':icer.life_exp1-icer.life_exp2,
            'cost difference (Euros)':icer.total_cost1-icer.total_cost2},ignore_index=True)

df_pivot = df_icer_sensitivity.pivot_table(index='param1',columns='param2',values='icer')
params = [x[2] for x in param_list]
#df_pivot = df_pivot.loc[params,params]

fig,ax = plt.subplots(figsize=(12,8))
sns.heatmap(ax=ax,data = df_pivot, annot=True, fmt=".0f")
plt.savefig(os.path.join(out_folder,'bivariate_'+strategy1+'vs'+strategy2+'.jpeg'))

In [ ]:
# A1 vs D



strategy1 = 'Strategy A1 (MS)'
strategy2 = 'Strategy D (ACMG)'

param_list = [
             ('testing_cost1',0.75*MS_cost,'MS cost -25%'),
              ('testing_cost1',1.25*MS_cost,'MS cost +25%'),
                 ('testing_cost2',0.75*ACMG_cost,'ACMG cost -25%'),
              ('testing_cost2',1.25*ACMG_cost,'ACMG cost +25%'),
              ('intervention_cost',1.25*intervention_cost,'Intervention cost +25%'),
              ('intervention_cost',0.75*intervention_cost,'Intervention cost -25%'),
                ('rho_notint',0.4,'Intervention rate 60%'),
              ('rho_notint',0.1,'Intervention rate 90%'),
             ('eps_cs',0.02,'ECS detection rate 98%'),
             ('eps_pgt',0.02,'Intervention success rate 98%'),
    ('original_model_fake_parameter',0,'Original Model')]


df_icer_sensitivity = pd.DataFrame()

for idx1 in range(len(param_list)):
    for idx2 in range(len(param_list)):
        
        param1 = param_list[idx1]
        param2 = param_list[idx2]
        #print(param1,param2)
        if (param1[1]!=param2[1])&(param1[0]==param2[0]):
            pass
        
        else:
        

            icer = Icer(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2],
                     eps_cs=0,eps_pgt=0,intervention_cost=intervention_cost,rho_notint=rho_notint)
            setattr(icer,param1[0],param1[1])
            setattr(icer,param2[0],param2[1])


            icer.compute_icer()
            df_icer_sensitivity = df_icer_sensitivity.append(
            {'param1':param1[2],'param1_value':param1[1],
             'param2':param2[2],'param2_value':param2[1],
               'icer':icer.icer,
               'total cost1':icer.total_cost1,
               'total cost2':icer.total_cost2,
               'life exp1':icer.life_exp1,
               'life exp2':icer.life_exp2,
              'benefit difference (years)':icer.life_exp1-icer.life_exp2,
            'cost difference (Euros)':icer.total_cost1-icer.total_cost2},ignore_index=True)

df_pivot = df_icer_sensitivity.pivot_table(index='param1',columns='param2',values='icer')
params = [x[2] for x in param_list]
#df_pivot = df_pivot.loc[params,params]

fig,ax = plt.subplots(figsize=(12,8))
sns.heatmap(ax=ax,data = df_pivot, annot=True, fmt=".0f")
plt.savefig(os.path.join(out_folder,'bivariate_'+strategy1+'vs'+strategy2+'.jpeg'))

In [ ]:
# A2 vs D




strategy1 = 'Strategy A (FS)'
strategy2 = 'Strategy D (ACMG)'

param_list = [
             ('testing_cost1',0.75*FS_cost,'FS cost -25%'),
              ('testing_cost1',1.25*FS_cost,'FS cost +25%'),
                 ('testing_cost2',0.75*ACMG_cost,'ACMG cost -25%'),
              ('testing_cost2',1.25*ACMG_cost,'ACMG cost +25%'),
              ('intervention_cost',1.25*intervention_cost,'Intervention cost +25%'),
              ('intervention_cost',0.75*intervention_cost,'Intervention cost -25%'),
                ('rho_notint',0.4,'Intervention rate 60%'),
              ('rho_notint',0.1,'Intervention rate 90%'),
             ('eps_cs',0.02,'ECS detection rate 98%'),
             ('eps_pgt',0.02,'Intervention success rate 98%'),
    ('original_model_fake_parameter',0,'Original Model')]


df_icer_sensitivity = pd.DataFrame()

for idx1 in range(len(param_list)):
    for idx2 in range(len(param_list)):
        
        param1 = param_list[idx1]
        param2 = param_list[idx2]
        #print(param1,param2)
        if (param1[1]!=param2[1])&(param1[0]==param2[0]):
            pass
        
        else:
        

            icer = Icer(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2],
                     eps_cs=0,eps_pgt=0,intervention_cost=intervention_cost,rho_notint=rho_notint)
            setattr(icer,param1[0],param1[1])
            setattr(icer,param2[0],param2[1])


            icer.compute_icer()
            df_icer_sensitivity = df_icer_sensitivity.append(
            {'param1':param1[2],'param1_value':param1[1],
             'param2':param2[2],'param2_value':param2[1],
               'icer':icer.icer,
               'total cost1':icer.total_cost1,
               'total cost2':icer.total_cost2,
               'life exp1':icer.life_exp1,
               'life exp2':icer.life_exp2,
              'benefit difference (years)':icer.life_exp1-icer.life_exp2,
            'cost difference (Euros)':icer.total_cost1-icer.total_cost2},ignore_index=True)

df_pivot = df_icer_sensitivity.pivot_table(index='param1',columns='param2',values='icer')
params = [x[2] for x in param_list]
#df_pivot = df_pivot.loc[params,params]

fig,ax = plt.subplots(figsize=(12,8))
sns.heatmap(ax=ax,data = df_pivot, annot=True, fmt=".0f")
plt.savefig(os.path.join(out_folder,'bivariate_'+strategy1+'vs'+strategy2+'.jpeg'))

In [ ]:
# A2 vs C

strategy1 = 'Strategy A (FS)'
strategy2 = 'Strategy C (ACOG)'

param_list = [
             ('testing_cost1',0.75*FS_cost,'FS cost -25%'),
              ('testing_cost1',1.25*FS_cost,'FS cost +25%'),
                 ('testing_cost2',0.75*ACOG_cost,'ACOG cost -25%'),
              ('testing_cost2',1.25*ACOG_cost,'ACOG cost +25%'),
              ('intervention_cost',1.25*intervention_cost,'Intervention cost +25%'),
              ('intervention_cost',0.75*intervention_cost,'Intervention cost -25%'),
                ('rho_notint',0.4,'Intervention rate 60%'),
              ('rho_notint',0.1,'Intervention rate 90%'),
             ('eps_cs',0.02,'ECS detection rate 98%'),
             ('eps_pgt',0.02,'Intervention success rate 98%'),
    ('original_model_fake_parameter',0,'Original Model')]


df_icer_sensitivity = pd.DataFrame()

for idx1 in range(len(param_list)):
    for idx2 in range(len(param_list)):
        
        param1 = param_list[idx1]
        param2 = param_list[idx2]
        #print(param1,param2)
        if (param1[1]!=param2[1])&(param1[0]==param2[0]):
            pass
        
        else:
        

            icer = Icer(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2],
                     eps_cs=0,eps_pgt=0,intervention_cost=intervention_cost,rho_notint=rho_notint)
            setattr(icer,param1[0],param1[1])
            setattr(icer,param2[0],param2[1])


            icer.compute_icer()
            df_icer_sensitivity = df_icer_sensitivity.append(
            {'param1':param1[2],'param1_value':param1[1],
             'param2':param2[2],'param2_value':param2[1],
               'icer':icer.icer,
               'total cost1':icer.total_cost1,
               'total cost2':icer.total_cost2,
               'life exp1':icer.life_exp1,
               'life exp2':icer.life_exp2,
              'benefit difference (years)':icer.life_exp1-icer.life_exp2,
            'cost difference (Euros)':icer.total_cost1-icer.total_cost2},ignore_index=True)

df_pivot = df_icer_sensitivity.pivot_table(index='param1',columns='param2',values='icer')
params = [x[2] for x in param_list]
#df_pivot = df_pivot.loc[params,params]

fig,ax = plt.subplots(figsize=(12,8))
sns.heatmap(ax=ax,data = df_pivot, annot=True, fmt=".0f")
plt.savefig(os.path.join(out_folder,'bivariate_'+strategy1+'vs'+strategy2+'.jpeg'))

In [ ]:
# A1 vs C


intervention_cost = 19714
rho_notint = 0.23
MS_cost = dl.strategy_dict['Strategy A1 (MS)'].testing_cost
ACOG_cost = dl.strategy_dict['Strategy C (ACOG)'].testing_cost

strategy1 = 'Strategy A1 (MS)'
strategy2 = 'Strategy C (ACOG)'

param_list = [
             ('testing_cost1',0.75*MS_cost,'MS cost -25%'),
              ('testing_cost1',1.25*MS_cost,'MS cost +25%'),
                 ('testing_cost2',0.75*ACOG_cost,'ACOG cost -25%'),
              ('testing_cost2',1.25*ACOG_cost,'ACOG cost +25%'),
              ('intervention_cost',1.25*intervention_cost,'Intervention cost +25%'),
              ('intervention_cost',0.75*intervention_cost,'Intervention cost -25%'),
                ('rho_notint',0.4,'Intervention rate 60%'),
              ('rho_notint',0.1,'Intervention rate 90%'),
             ('eps_cs',0.02,'ECS detection rate 98%'),
             ('eps_pgt',0.02,'Intervention success rate 98%'),
    ('original_model_fake_parameter',0,'Original Model')]


df_icer_sensitivity = pd.DataFrame()

for idx1 in range(len(param_list)):
    for idx2 in range(len(param_list)):
        
        param1 = param_list[idx1]
        param2 = param_list[idx2]
        #print(param1,param2)
        if (param1[1]!=param2[1])&(param1[0]==param2[0]):
            pass
        
        else:
        

            icer = Icer(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2],
                     eps_cs=0,eps_pgt=0,intervention_cost=intervention_cost,rho_notint=rho_notint)
            setattr(icer,param1[0],param1[1])
            setattr(icer,param2[0],param2[1])


            icer.compute_icer()
            df_icer_sensitivity = df_icer_sensitivity.append(
            {'param1':param1[2],'param1_value':param1[1],
             'param2':param2[2],'param2_value':param2[1],
               'icer':icer.icer,
               'total cost1':icer.total_cost1,
               'total cost2':icer.total_cost2,
               'life exp1':icer.life_exp1,
               'life exp2':icer.life_exp2,
              'benefit difference (years)':icer.life_exp1-icer.life_exp2,
            'cost difference (Euros)':icer.total_cost1-icer.total_cost2},ignore_index=True)

df_pivot = df_icer_sensitivity.pivot_table(index='param1',columns='param2',values='icer')
params = [x[2] for x in param_list]
#df_pivot = df_pivot.loc[params,params]

fig,ax = plt.subplots(figsize=(12,8))
sns.heatmap(ax=ax,data = df_pivot, annot=True, fmt=".0f")
plt.savefig(os.path.join(out_folder,'bivariate_'+strategy1+'vs'+strategy2+'.jpeg'))

In [ ]:
# B vs no testing


strategy1 = 'Strategy B (ACOG+ACMG)'
strategy2 = 'No testing'

param_list = [
             ('testing_cost1',0.75*ACOGACMG_cost,'Testing cost -25%'),
              ('testing_cost1',1.25*ACOGACMG_cost,'Testing cost +25%'),
              ('intervention_cost',1.25*intervention_cost,'Intervention cost +25%'),
              ('intervention_cost',0.75*intervention_cost,'Intervention cost -25%'),
                ('rho_notint',0.4,'Intervention rate 60%'),
              ('rho_notint',0.1,'Intervention rate 90%'),
             ('eps_cs',0.02,'ECS detection rate 98%'),
             ('eps_pgt',0.02,'Intervention success rate 98%'),
    ('original_model_fake_parameter',0,'Original Model')]


df_icer_sensitivity = pd.DataFrame()

for idx1 in range(len(param_list)):
    for idx2 in range(len(param_list)):
        
        param1 = param_list[idx1]
        param2 = param_list[idx2]
        #print(param1,param2)
        if (param1[1]!=param2[1])&(param1[0]==param2[0]):
            pass
        
        else:
        

            icer = Icer(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2],
                     eps_cs=0,eps_pgt=0,intervention_cost=intervention_cost,rho_notint=rho_notint)
            setattr(icer,param1[0],param1[1])
            setattr(icer,param2[0],param2[1])


            icer.compute_icer()
            df_icer_sensitivity = df_icer_sensitivity.append(
            {'param1':param1[2],'param1_value':param1[1],
             'param2':param2[2],'param2_value':param2[1],
               'icer':icer.icer,
               'total cost1':icer.total_cost1,
               'total cost2':icer.total_cost2,
               'life exp1':icer.life_exp1,
               'life exp2':icer.life_exp2,
              'benefit difference (years)':icer.life_exp1-icer.life_exp2,
            'cost difference (Euros)':icer.total_cost1-icer.total_cost2},ignore_index=True)

df_pivot = df_icer_sensitivity.pivot_table(index='param1',columns='param2',values='icer')
params = [x[2] for x in param_list]
#df_pivot = df_pivot.loc[params,params]

fig,ax = plt.subplots(figsize=(12,8))
sns.heatmap(ax=ax,data = df_pivot, annot=True, fmt=".0f")
plt.savefig(os.path.join(out_folder,'bivariate_'+strategy1+'vs'+strategy2+'.jpeg'))

In [ ]:
# A2 vs B

strategy1 = 'Strategy A (FS)'
strategy2 = 'Strategy B (ACOG+ACMG)'

param_list = [
             ('testing_cost1',0.75*FS_cost,'FS cost -25%'),
              ('testing_cost1',1.25*FS_cost,'FS cost +25%'),
                 ('testing_cost2',0.75*ACOGACMG_cost,'ACOG+ACMG cost -25%'),
              ('testing_cost2',1.25*ACOGACMG_cost,'ACOG+ACMG cost +25%'),
              ('intervention_cost',1.25*intervention_cost,'Intervention cost +25%'),
              ('intervention_cost',0.75*intervention_cost,'Intervention cost -25%'),
                ('rho_notint',0.4,'Intervention rate 60%'),
              ('rho_notint',0.1,'Intervention rate 90%'),
             ('eps_cs',0.02,'ECS detection rate 98%'),
             ('eps_pgt',0.02,'Intervention success rate 98%'),
    ('original_model_fake_parameter',0,'Original Model')]


df_icer_sensitivity = pd.DataFrame()

for idx1 in range(len(param_list)):
    for idx2 in range(len(param_list)):
        
        param1 = param_list[idx1]
        param2 = param_list[idx2]
        #print(param1,param2)
        if (param1[1]!=param2[1])&(param1[0]==param2[0]):
            pass
        
        else:
        

            icer = Icer(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2],
                     eps_cs=0,eps_pgt=0,intervention_cost=intervention_cost,rho_notint=rho_notint)
            setattr(icer,param1[0],param1[1])
            setattr(icer,param2[0],param2[1])


            icer.compute_icer()
            df_icer_sensitivity = df_icer_sensitivity.append(
            {'param1':param1[2],'param1_value':param1[1],
             'param2':param2[2],'param2_value':param2[1],
               'icer':icer.icer,
               'total cost1':icer.total_cost1,
               'total cost2':icer.total_cost2,
               'life exp1':icer.life_exp1,
               'life exp2':icer.life_exp2,
              'benefit difference (years)':icer.life_exp1-icer.life_exp2,
            'cost difference (Euros)':icer.total_cost1-icer.total_cost2},ignore_index=True)

df_pivot = df_icer_sensitivity.pivot_table(index='param1',columns='param2',values='icer')
params = [x[2] for x in param_list]
#df_pivot = df_pivot.loc[params,params]

fig,ax = plt.subplots(figsize=(12,8))
sns.heatmap(ax=ax,data = df_pivot, annot=True, fmt=".0f")
plt.savefig(os.path.join(out_folder,'bivariate_'+strategy1+'vs'+strategy2+'.jpeg'))

# Probabilistic analysis

In [ ]:

strategy1 = 'Strategy A1 (MS)'
strategy2 = 'Strategy D (ACMG)'

psa = Psa(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2])
psa.run_mc(
    intervention_cost_lb=intervention_cost*0.75, intervention_cost_ub=intervention_cost*1.25,
       testing_cost1_lb=0.75*MS_cost, testing_cost1_ub=1.25*MS_cost,
        testing_cost2_lb=0.75*AMCG_cost, testing_cost2_ub=1.25*AMCG_cost,
        eps_cs_lb=0,eps_cs_ub=0.02,
        eps_pgt_lb=0,eps_pgt_ub=0.02,
        rho_notint_lb=0.1,rho_notint_ub=0.4,nsim=10000)



outfile = os.path.join(out_folder,'psa_'+strategy1+'vs'+strategy2+'.jpeg')

psa.plot_ceac_hist(outfile=outfile)

In [ ]:

strategy1 = 'Strategy A (FS)'
strategy2 = 'Strategy B (ACOG+ACMG)'

psa = Psa(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2])
psa.run_mc(
    intervention_cost_lb=intervention_cost*0.75, intervention_cost_ub=intervention_cost*1.25,
       testing_cost1_lb=0.75*FS_cost, testing_cost1_ub=1.25*FS_cost,
        testing_cost2_lb=0.75*ACOGACMG_cost, testing_cost2_ub=1.25*ACOGACMG_cost,
        eps_cs_lb=0,eps_cs_ub=0.02,
        eps_pgt_lb=0,eps_pgt_ub=0.02,
        rho_notint_lb=0.1,rho_notint_ub=0.4,nsim=10000)



outfile = os.path.join(out_folder,'psa_'+strategy1+'vs'+strategy2+'.jpeg')

psa.plot_ceac_hist(outfile=outfile)

In [ ]:

strategy1 = 'Strategy A (FS)'
strategy2 = 'No testing'

psa = Psa(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2])
psa.run_mc(
    intervention_cost_lb=intervention_cost*0.75, intervention_cost_ub=intervention_cost*1.25,
       testing_cost1_lb=0.75*FS_cost, testing_cost1_ub=1.25*FS_cost,
        testing_cost2_lb=0, testing_cost2_ub=0,
        eps_cs_lb=0,eps_cs_ub=0.02,
        eps_pgt_lb=0,eps_pgt_ub=0.02,
        rho_notint_lb=0.1,rho_notint_ub=0.4,nsim=10000)



outfile = os.path.join(out_folder,'psa_'+strategy1+'vs'+strategy2+'.jpeg')

psa.plot_ceac_hist(outfile=outfile)

In [ ]:

strategy1 = 'Strategy B (ACOG+ACMG)'
strategy2 = 'No testing'

psa = Psa(dl.strategy_dict[strategy1],dl.strategy_dict[strategy2])
psa.run_mc(
    intervention_cost_lb=intervention_cost*0.75, intervention_cost_ub=intervention_cost*1.25,
       testing_cost1_lb=0.75*ACOGACMG_cost, testing_cost1_ub=1.25*ACOGACMG_cost,
        testing_cost2_lb=0, testing_cost2_ub=0,
        eps_cs_lb=0,eps_cs_ub=0.02,
        eps_pgt_lb=0,eps_pgt_ub=0.02,
        rho_notint_lb=0.1,rho_notint_ub=0.4,nsim=10000)



outfile = os.path.join(out_folder,'psa_'+strategy1+'vs'+strategy2+'.jpeg')

psa.plot_ceac_hist(outfile=outfile)

In [ ]:
psa.plot_ceac()